# Exercise 5: Vanilla Policy Gradient

In [ ]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.python.framework import ops
import numpy as np
import gym
import matplotlib.pyplot as plt
import os
%matplotlib inline

from unityenv import UnityEnvironment

### Load the environment

In [ ]:
env_config = {}
env = UnityEnvironment("physics", True, 0, False, env_config)
print str(env)

### The Actor Critic Agent

In [ ]:
gamma = 0.99

def discount_rewards(r):
    #r = (r - np.mean(r)) / (np.std(r) + 1e-10)
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(xrange(0, r.size)):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

In [ ]:
class ACAgent():
    def __init__(self, lr, s_size,a_size,h_size):
        #These lines established the feed-forward part of the network. The agent takes a state and produces an action.
        self.state_in= tf.placeholder(shape=[None,s_size],dtype=tf.float32)
        self.batch_size = tf.placeholder(shape=None, dtype=tf.int32)
        hidden = slim.fully_connected(self.state_in,h_size,biases_initializer=None,activation_fn=tf.nn.elu)
        self.out = slim.fully_connected(hidden,a_size,activation_fn=tf.nn.softmax,biases_initializer=None)
        self.output = self.out * (0.9) + 0.1/a_size
        self.chosen_action = tf.argmax(self.output,1)

        #The next six lines establish the training proceedure. We feed the reward and chosen action into the network
        #to compute the loss, and use it to update the network.
        self.reward_holder = tf.placeholder(shape=[None],dtype=tf.float32)
        self.action_holder = tf.placeholder(shape=[None],dtype=tf.int32)
        self.actions = slim.one_hot_encoding(self.action_holder, a_size)
        
        self.responsible_outputs = tf.reduce_sum(self.output * self.actions, axis=1)
        #self.advantage = self.reward_holder
        
        self.loss = -tf.reduce_mean(tf.log(self.responsible_outputs)*self.advantage)
        
        tvars = tf.trainable_variables()
        self.gradient_holders = []
        for idx,var in enumerate(tvars):
            placeholder = tf.placeholder(tf.float32,name=str(idx)+'_holder')
            self.gradient_holders.append(placeholder)
        
        self.gradients = tf.gradients(self.loss,tvars)
        
        optimizer = tf.train.AdamOptimizer(learning_rate=lr)
        self.update_batch = optimizer.apply_gradients(zip(self.gradient_holders,tvars))

### Training the Agent

In [ ]:
tf.reset_default_graph() #Clear the Tensorflow graph.

myAgent = RelationalAgent(lr=3e-4,s_size=env.state_space_size,a_size=env.action_space_size,h_size=128, num_objects=5, num_features=5) #Load the agent.

if not os.path.exists('./summary'):
    os.makedirs('./summary')

total_episodes = 20000 #Set total number of episodes to train agent on.
max_ep = 999
update_frequency = 10

init = tf.global_variables_initializer()

# Launch the tensorflow graph
with tf.Session() as sess:
    sess.run(init)
    summary_writer = tf.summary.FileWriter('./summary')
    i = 0
    total_reward = []
    total_lenght = []
        
    gradBuffer = sess.run(tf.trainable_variables())
    for ix,grad in enumerate(gradBuffer):
        gradBuffer[ix] = grad * 0
        
    while i < total_episodes:
        o, s = env.reset()
        running_reward = 0
        ep_history = []
        for j in range(max_ep):
            #Probabilistically pick an action given our network outputs.
            a_dist, io = sess.run([myAgent.output, myAgent.input_objects],feed_dict={myAgent.state_in:[s], myAgent.batch_size: 1})
            a = np.random.choice(a_dist[0],p=a_dist[0])
            a = np.argmax(a_dist == a)

            o1, s1,r,d = env.step(a, 0) #Get our reward for taking an action given a bandit.
            ep_history.append([s,a,r,s1])
            s = s1
            running_reward += r
            if d == True:
                #Update the network.
                ep_history = np.array(ep_history)
                ep_history[:,2] = discount_rewards(ep_history[:,2])
                feed_dict={myAgent.reward_holder:ep_history[:,2],
                        myAgent.action_holder:ep_history[:,1],
                        myAgent.state_in:np.vstack(ep_history[:,0]),
                        myAgent.batch_size: len(ep_history)}
                p_loss, grads = sess.run([myAgent.loss, myAgent.gradients], feed_dict=feed_dict)
                for idx,grad in enumerate(grads):
                    gradBuffer[idx] += grad

                if i % update_frequency == 0 and i != 0:
                    feed_dict = dictionary = dict(zip(myAgent.gradient_holders, gradBuffer))
                    _ = sess.run(myAgent.update_batch, feed_dict=feed_dict)
                    for ix,grad in enumerate(gradBuffer):
                        gradBuffer[ix] = grad * 0
                
                total_reward.append(running_reward)
                total_lenght.append(j)
                break

        
            #Update our running tally of scores.
        if i % 50 == 0 and i != 0:
            summary = tf.Summary()
            summary.value.add(tag='Info/Reward', simple_value=float(np.mean(total_reward[-50:])))
            summary.value.add(tag='Info/Policy Loss', simple_value=float(p_loss))

            summary_writer.add_summary(summary, i)
            summary_writer.flush()
            print(np.mean(total_reward[-50:]))
        i += 1

In [ ]:
env.close()